<a href="https://colab.research.google.com/github/Tristan-Brown1096/DS-Unit-2-Kaggle-Challenge/blob/master/module4-classification-metrics/LS_DS_224_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 4*

---

# Classification Metrics

## Assignment
- [ ] If you haven't yet, [review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Plot a confusion matrix for your Tanzania Waterpumps model.
- [ ] Continue to participate in our Kaggle challenge. Every student should have made at least one submission that scores at least 70% accuracy (well above the majority class baseline).
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_
- [ ] Commit your notebook to your fork of the GitHub repo.
- [ ] Read [Maximizing Scarce Maintenance Resources with Data: Applying predictive modeling, precision at k, and clustering to optimize impact](http://archive.is/DelgE), by Lambda DS3 student Michael Brady. His blog post extends the Tanzania Waterpumps scenario, far beyond what's in the lecture notebook.


## Stretch Goals

### Reading

- [Attacking discrimination with smarter machine learning](https://research.google.com/bigpicture/attacking-discrimination-in-ml/), by Google Research, with  interactive visualizations. _"A threshold classifier essentially makes a yes/no decision, putting things in one category or another. We look at how these classifiers work, ways they can potentially be unfair, and how you might turn an unfair classifier into a fairer one. As an illustrative example, we focus on loan granting scenarios where a bank may grant or deny a loan based on a single, automatically computed number such as a credit score."_
- [Notebook about how to calculate expected value from a confusion matrix by treating it as a cost-benefit matrix](https://github.com/podopie/DAT18NYC/blob/master/classes/13-expected_value_cost_benefit_analysis.ipynb)
- [Visualizing Machine Learning Thresholds to Make Better Business Decisions](https://blog.insightdatascience.com/visualizing-machine-learning-thresholds-to-make-better-business-decisions-4ab07f823415)


### Doing
- [ ] Share visualizations in our Slack channel!
- [ ] RandomizedSearchCV / GridSearchCV, for model selection. (See module 3 assignment notebook)
- [ ] Stacking Ensemble. (See module 3 assignment notebook)
- [ ] More Categorical Encoding. (See module 2 assignment notebook)

In [6]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [7]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [8]:
train.date_recorded = pd.to_datetime(train.date_recorded)
test.date_recorded = pd.to_datetime(test.date_recorded)

In [9]:
import numpy as np
train.construction_year = train.construction_year.replace(0, np.nan)
test.construction_year = test.construction_year.replace(0, np.nan)

In [10]:
train['age_at_inspection'] = train['date_recorded'].dt.year - train['construction_year']
test['age_at_inspection'] = test['date_recorded'].dt.year - test['construction_year']

In [11]:
train['pop_served'] = train.age_at_inspection * train.population

In [12]:
def wrangle(X):
  #copy X
  X = X.copy()

  #replace weird latitude values
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  #replace lat/long 0 values with NaN
  cols_with_zeros = ['longitude', 'latitude']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)

  #drop constant columns
  X = X.drop([col for col in X if X[col].nunique()==1], axis=1)

  #drop high cardinality columns
  hc = [col for col in X.describe(include='object').columns if X[col].nunique() > 100]
  X = X.drop(hc, axis=1)

  #drop repeated column
  X = X.drop(['quantity_group', 'date_recorded', 'construction_year', 'population', 'age_at_inspection'], axis=1)

  #return new version 
  return X

In [13]:
train = wrangle(train)
test = wrangle(test)

#Split data

In [14]:
y = train['status_group']

X = train.drop('status_group', axis=1)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 42)

#Build model_27

In [17]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
model_27 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_features='sqrt', max_depth=20)
)

model_27.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='sqrt',
                                   

In [18]:
print('Training accuracy', model_27.score(X_train, y_train))
print('Validation accuracy', model_27.score(X_val, y_val))

Training accuracy 0.9497053872053872
Validation accuracy 0.805050505050505


#Build model_28

In [20]:
model_28 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=400, max_features='auto', max_depth=19)
    
)

model_28.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['basin', 'region', 'public_meeting',
                                      'scheme_management', 'permit',
                                      'extraction_type',
                                      'extraction_type_group',
                                      'extraction_type_class', 'management',
                                      'management_group', 'payment',
                                      'payment_type', 'water_quality',
                                      'quality_group', 'quantity', 'source',
                                      'source_type', 'source_class',
                                      'waterpoint_type...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=19, max_features='auto',
                                   

In [21]:
print('Training accuracy', model_28.score(X_train, y_train))
print('Validation accuracy', model_28.score(X_val, y_val))

Training accuracy 0.9367003367003367
Validation accuracy 0.8056397306397306


#Build model_29

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [29]:
scaler = StandardScaler()
oc = OrdinalEncoder()
imputer = SimpleImputer()
pca = PCA(.9)

In [31]:
X_train_pca = pca.fit_transform(scaler.fit_transform(imputer.fit_transform(oc.fit_transform(X_train))))
X_val_pca = pca.fit_transform(scaler.fit_transform(imputer.fit_transform(oc.fit_transform(X_val))))

In [33]:
model_29 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=400, max_features='auto', max_depth=19)
    
)

model_29.fit(X_train_pca, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[], return_df=True, verbose=0)),
                ('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('randomforestclassifier',
                 RandomForestClassifier(boo...ap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=19, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                         

In [34]:
print('Training accuracy', model_29.score(X_train_pca, y_train))
print('Validation accuracy', model_29.score(X_val_pca, y_val))

Training accuracy 0.9441919191919191
Validation accuracy 0.732912457912458


#Build model_30

In [36]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [39]:
params = {'randomforestclassifier__n_estimators': range(300, 600, 20),
          'randomforestclassifier__max_depth': range(14, 21),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_27, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [40]:
gs.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import plot_confusion_matrix, classification_report

In [ ]:
plot_confusion_matrix(pipeline, X_val, y_val, 
                      values_format=".0f", 
                      display_labels=['no repair needed', 'repair needed'])

In [ ]:
# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']
 
target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]
 
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)